In [ ]:
import MySQLdb
import sshtunnel
from config import config

sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    config['remote_machine']['hostname'],
    ssh_username=config['remote_machine']['ssh_username'], 
    ssh_password=config['remote_machine']['ssh_password'],
    remote_bind_address=(config['remote_machine']['msql_hostname'], 3306)
) as tunnel:
    con = MySQLdb.connect(
        user=config['mysql_dev']['user'],
        passwd=config['mysql_dev']['passwd'],
        host='127.0.0.1', #have to set this exact value
        port=tunnel.local_bind_port,
        db=config['mysql_dev']['db'],
    )
    # Do stuff
    query = """INSERT INTO youtube_comment_stg (target_link, title, user_id, user_name, comment)
    VALUES (%s, %s, %s, %s, %s);
    """
    cursor = con.cursor() #Connection.cursor() return Cursor object
    #prepare data for insert
    data = data.replace(r'^\s*$', pd.np.nan, regex=True)
    data = data.where((pd.notnull(data)), None)
    #params have to be list of tuple for executemany, each tuple is a row
    #params have to be tuple for execute.
    data = [tuple(row) for row in data.values] 
    try: 
        cursor.executemany(query, data)
        con.commit()
    except Exception as e:
        print('Error: ', e)
        con.rollback()
        raise Exception(e)
    data = pd.DataFrame(data)
    con.close()